In [1]:
import numpy as np
import torch

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [2]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [3]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [4]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.8474,  1.0004,  0.9559],
        [ 0.3555,  1.1560,  0.3668]], requires_grad=True)
tensor([ 0.2883, -0.1208], requires_grad=True)


In [5]:
def model(x):
    return x @ w.t() + b

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables

In [6]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 46.5584, 119.0588],
        [ 72.3873, 157.4382],
        [116.0605, 206.9931],
        [ -7.7614,  99.4219],
        [104.7687, 161.0669]], grad_fn=<AddBackward0>)


In [7]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

Calculate the difference between the two matrices (preds and targets).
Square all elements of the difference matrix to remove negative values.
Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [8]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

torch.sum returns the sum of all the elements in a tensor. The .numel method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [9]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(1779.3975, grad_fn=<DivBackward0>)


Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [10]:
# Compute gradients
loss.backward()

In [11]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.8474,  1.0004,  0.9559],
        [ 0.3555,  1.1560,  0.3668]], requires_grad=True)
tensor([[-928.4713, -578.8688, -421.0116],
        [4884.8447, 4978.2300, 3053.4944]])


In [12]:
w
w.grad

tensor([[-928.4713, -578.8688, -421.0116],
        [4884.8447, 4978.2300, 3053.4944]])

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [13]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [14]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(1779.3975, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [15]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [16]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 47.8051, 110.8439],
        [ 74.0111, 146.6574],
        [117.8883, 194.3009],
        [ -6.4096,  91.1683],
        [106.2599, 150.7793]], grad_fn=<AddBackward0>)


In [17]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(1238.9717, grad_fn=<DivBackward0>)


In [18]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-800.7562, -444.3427, -337.4223],
        [4037.5085, 4069.7515, 2492.5176]])
tensor([-8.2890, 46.7500])


In [19]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [20]:
print(w)
print(b)

tensor([[-0.8301,  1.0106,  0.9635],
        [ 0.2663,  1.0656,  0.3114]], requires_grad=True)
tensor([ 0.2884, -0.1219], requires_grad=True)


In [21]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(874.3945, grad_fn=<DivBackward0>)


Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [22]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(52.1701, grad_fn=<DivBackward0>)


In [24]:
# Predictions
preds

tensor([[ 55.1244,  72.0918],
        [ 82.5111,  96.8825],
        [121.2830, 138.7065],
        [  9.1087,  46.9952],
        [109.4936, 106.7000]], grad_fn=<AddBackward0>)

In [25]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [27]:
import torch.nn as nn

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [28]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [29]:
from torch.utils.data import TensorDataset

# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [30]:
from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [31]:
#We can use the data loader in a for loop. Let's look at an example.
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 92.,  87.,  64.],
        [ 91.,  87.,  65.],
        [103.,  43.,  36.],
        [ 68.,  96.,  71.],
        [ 87., 135.,  57.]])
tensor([[ 82., 100.],
        [ 80., 102.],
        [ 20.,  38.],
        [104., 118.],
        [118., 134.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [32]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1338,  0.1771, -0.2599],
        [ 0.4186,  0.1400,  0.5511]], requires_grad=True)
Parameter containing:
tensor([ 0.0577, -0.0358], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [33]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1338,  0.1771, -0.2599],
         [ 0.4186,  0.1400,  0.5511]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0577, -0.0358], requires_grad=True)]

In [34]:
# Generate predictions
preds = model(inputs)
preds

tensor([[10.5178, 63.6002],
        [11.1881, 85.6484],
        [20.3588, 87.1077],
        [11.7079, 69.0730],
        [ 8.1012, 80.8658],
        [10.4745, 63.8788],
        [10.7511, 86.0595],
        [20.2328, 88.0774],
        [11.7512, 68.7944],
        [ 7.7075, 80.9983],
        [10.0808, 64.0113],
        [11.1448, 85.9270],
        [20.7958, 86.6966],
        [12.1016, 68.9405],
        [ 8.1445, 80.5872]], grad_fn=<AddmmBackward0>)

Loss Function

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [35]:
# Import nn.functional
import torch.nn.functional as F

In [36]:
# Define loss function
loss_fn = F.mse_loss

In [37]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(3052.1982, grad_fn=<MseLossBackward0>)


Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [38]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [39]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [40]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 649.8585
Epoch [20/100], Loss: 280.5393
Epoch [30/100], Loss: 531.8451
Epoch [40/100], Loss: 112.3054
Epoch [50/100], Loss: 175.2038
Epoch [60/100], Loss: 131.9217
Epoch [70/100], Loss: 54.7518
Epoch [80/100], Loss: 74.0696
Epoch [90/100], Loss: 66.8988
Epoch [100/100], Loss: 23.3879


In [41]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.4413,  71.5904],
        [ 79.0725, 100.2261],
        [122.6772, 131.3743],
        [ 28.9076,  44.7930],
        [ 91.9910, 113.8574],
        [ 57.2969,  70.6861],
        [ 78.3042, 100.1948],
        [122.6767, 131.9577],
        [ 30.0520,  45.6973],
        [ 92.3672, 114.7304],
        [ 57.6731,  71.5591],
        [ 77.9281,  99.3217],
        [123.4454, 131.4056],
        [ 28.5315,  43.9199],
        [ 93.1354, 114.7617]], grad_fn=<AddmmBackward0>)

In [42]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [43]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.4282, 68.9909]], grad_fn=<AddmmBackward0>)